In [10]:
import pandas as pd
import numpy as np
from glob import glob
import os
import json
from PIL import Image
import albumentations as A
import random
import os
from os.path import exists
#import glob
import json
import shutil
import random 
import cv2

In [2]:
fals_lumbar = glob("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/fals-lumbar/*")

In [3]:
len(fals_lumbar)

6

In [4]:
sals_lumbar = glob("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/*")

In [5]:
len(sals_lumbar)

9

In [6]:
num_elements_to_select = 2
random_sample = random.sample(fals_lumbar, num_elements_to_select)
random_sample = ['/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/fals-lumbar/NEUKZ727YY5_Lumbar_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/fals-lumbar/NEUNY116EZW_Spinal_Cord_Lumbar_pTDP_43']

In [7]:
num_elements_to_select = 3
random_sample2 = random.sample(sals_lumbar, num_elements_to_select)
random_sample2 = ['/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUDK592BDZ_Lumbosacral_Spinal_Cord_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUCP856ZH7_Spinal_Cord_Lumbar_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUEL133AK6_Lumbar_pTDP_43']

In [8]:
train_samples = [x for x in fals_lumbar if x not in random_sample]
print(len(train_samples))
train_samples2 = [x for x in sals_lumbar if x not in random_sample2]
print(len(train_samples2))
train_samples.extend(train_samples2)

4
6


In [9]:
len(train_samples)

10

In [41]:
def filter_masks(mask_path):
    mask = np.array(Image.open(mask_path).convert("P"))
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_with_area = [(c, cv2.contourArea(c)) for c in contours]
    selected_contours = [k for k in contour_with_area if k[1]>2000]
    if len(selected_contours)>=1:
        return False
    else:
        return True

In [51]:
# Check count of labels on folders
all_images = []
all_masks = []

for file in train_samples:
    all_images.extend(glob(os.path.join(file,'image/*.png')))
    all_masks.extend(glob(os.path.join(file,'mask/*.png')))
    

#all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/*/*/image",'*.png'))
#all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/*/*/mask",'*.png'))

all_images.sort()
all_masks.sort()


pre_images, pre_masks, neg_images, neg_masks = [],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    filter_flag = filter_masks(mask)
    if filter_flag==False:
        if 100 in found:
            pre_images.append(image)
            pre_masks.append(mask)
        if 200 in found:
            neg_images.append(image)
            neg_masks.append(mask)

In [52]:
len(all_images)

466

In [53]:
len(pre_images)

172

In [55]:
len(neg_images)

221

In [57]:
# Use transformations for image
transforms  = A.Compose([ A.VerticalFlip(p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.Blur(blur_limit=1,p=0.2),
                            A.RandomRotate90(),
                            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
                            A.AdvancedBlur(p=0.2)
                        ])

# select files randomly
def get_randimages_dataug(total_imgs, image_filenames, label_filenames):
    image_filenames.sort()
    label_filenames.sort()
    random_image_file = []
    random_label_file = []
    for i in range(total_imgs):
        random.seed(i)
        random_image_file.append(random.choice(image_filenames))
        random.seed(i)
        random_label_file.append(random.choice(label_filenames))
    return [random_image_file, random_label_file]

# upsample dataset with the augmentations
def upsample_dataset(dataset_base_dir, random_img_filenames, rand_label_filenames, variations, transforms, dest_img_folder_name, dest_label_folder_name):
    i = 0
    aug_img_files = []
    aug_mask_files = []

    # Make dir where tha augmented file will reside
    aug_img_dir = os.path.join(dataset_base_dir, dest_img_folder_name)
    if not os.path.exists(aug_img_dir):
            os.makedirs(aug_img_dir)
            print("Augmented Directory '%s' created" %aug_img_dir)
    
    aug_mask_dir = os.path.join(dataset_base_dir, dest_label_folder_name)
    if not os.path.exists(aug_mask_dir):
            os.makedirs(aug_mask_dir)
            print("Augmented Directory '%s' created" %aug_mask_dir)

    print(os.listdir(dataset_base_dir))
    print("\nData Augmentation in Progress ...")
    total_imgs = len(random_img_filenames)
    
    for i in range(total_imgs):
        # load the image
        img = Image.open(random_img_filenames[i]).convert("RGB")
        img = np.array(img)
        mask = Image.open(rand_label_filenames[i]).convert('P')
        mask = np.array(mask)

        for j in range(variations):
                transformed = transforms(image=img, mask=mask)
                transformed_img = transformed["image"]
                transformed_img = Image.fromarray(transformed_img)

                #To rename the file with prefix A_
                filename = os.path.basename(random_img_filenames[i])
                filepath = os.path.dirname(random_img_filenames[i])

                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir,dest_img_folder_name,
                                        aug_file_name)
                transformed_img.save(new_file)
                aug_img_files.append(new_file)

                transformed_mask = transformed["mask"]
                transformed_mask = Image.fromarray(transformed_mask)

                #To rename the file with prefix A_
                filename = os.path.basename(rand_label_filenames[i])
                filepath = os.path.dirname(rand_label_filenames[i])
                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir, dest_label_folder_name,
                                        aug_file_name)
                transformed_mask.save(new_file)
                aug_mask_files.append(new_file)
    return aug_img_files, aug_mask_files

In [58]:
aug_value = 2000
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, pre_images, pre_masks)
#dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train"
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train_filtered"

aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

Augmented Directory '/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train_filtered/images' created
Augmented Directory '/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train_filtered/labels' created
['labels', 'images']

Data Augmentation in Progress ...


In [59]:
aug_value = 2000
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, neg_images, neg_masks)
#dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train"
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data//monika/ALS/training_dataset/train_filtered"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', 'images']

Data Augmentation in Progress ...


## VAL data

In [60]:
# Check count of labels on folders
all_images = []
all_masks = []


val_samples = ['/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/fals-lumbar/NEUKZ727YY5_Lumbar_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/fals-lumbar/NEUNY116EZW_Spinal_Cord_Lumbar_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUDK592BDZ_Lumbosacral_Spinal_Cord_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUCP856ZH7_Spinal_Cord_Lumbar_pTDP_43',
 '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/all_crops/sals-lumbar/NEUEL133AK6_Lumbar_pTDP_43']

print(len(val_samples))

for file in val_samples:
    all_images.extend(glob(os.path.join(file,'image/*.png')))
    all_masks.extend(glob(os.path.join(file,'mask/*.png')))

all_images.sort()
all_masks.sort()

pre_images, pre_masks, neg_images, neg_masks = [],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    filter_flag = filter_masks(mask)
    if filter_flag==False:
        if 100 in found:
            pre_images.append(image)
            pre_masks.append(mask)
        if 200 in found:
            neg_images.append(image)
            neg_masks.append(mask)

5


In [61]:
len(pre_images)

178

In [62]:
len(neg_images)

48

In [63]:
aug_value = 500
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, pre_images, pre_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/training_dataset/val_filtered"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

Augmented Directory '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/training_dataset/val_filtered/images' created
Augmented Directory '/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/training_dataset/val_filtered/labels' created
['labels', 'images']

Data Augmentation in Progress ...


In [64]:
aug_value = 500
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, neg_images, neg_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/ALS/training_dataset/val_filtered"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', 'images']

Data Augmentation in Progress ...
